In [2]:
import os
import json
import glob


In [52]:

class RawDataReader:
    def __init__(self, file_path):
        with open(file_path, 'r') as file:
            self.data = json.load(file)

        self.id = self.data['ID']
        self.file_identifier = self.data['patients_file_identifier']
        self.plan_details = self.extract_plan_details()

    def extract_plan_details(self):
        plan_details = []
        for plan in self.data['plans']:
            if plan['completed'] == True:
                plan_name = plan['name']
                planning_exam_name = plan['planning_examination']['name']
                planningCT_filename = plan['planning_examination']['dose_DICOM_filename']

                evaluation_exams = []
                for exam in plan['evaluation_examinations']:
                    for opt_case in exam['optimization_cases']:
                        if opt_case.get('completed', True) and 'gradient_descent_result' in opt_case:
                            final_trans_coord = opt_case['gradient_descent_result'].get('final_translation_coordinate', {})
                            x = round(final_trans_coord.get('x', 0), 5)
                            y = round(final_trans_coord.get('y', 0), 5)
                            z = round(final_trans_coord.get('z', 0), 5)
                            exam_detail = {
                                # 'name': exam['name'],
                                'final_translation_coordinate': {'x': x, 'y': y, 'z': z},
                                'repeatedCT_filename': opt_case['gradient_descent_result'].get('dose_DICOM_filename', '')
                            }
                            evaluation_exams.append(exam_detail)

                plan_details.append({
                    'plan_name': plan_name,
                    'planning_exam_name': planning_exam_name,
                    'planningCT_filename': planningCT_filename,
                    'evaluation_examinations': evaluation_exams
                })
            else:
                plan_details.append({
                    'plan_name': plan_name,
                    'planning_exam_name': None,
                    'planningCT_filename': None,
                    'evaluation_examinations': None
                })

        return plan_details

    def get_plan_details(self):
        return self.plan_details
    
# # Usage as an example
# file_path = '/home/shahpouriz/Data/DBP_newDATA/DBP/results_DBP_OP007_Photons_20231024_053647.json'
# treatment_data = RawDataReader (file_path)
# print(treatment_data.id)
# print(treatment_data.file_identifier)
# plan_details = treatment_data.get_plan_details()

# plan_details[0].get('plan_name')
# plan_details[0].get('planningCT_filename')
# plan_details[0].get('evaluation_examinations')

--------------------

In [53]:

class DirectoryProcessor:
    def __init__(self, directory_path):
        self.directory_path = directory_path

    def process_files(self):
        
        # Search for json files matching the pattern
        file_pattern = os.path.join(self.directory_path, 'results_DBP_OP*.json')
        json_files = glob.glob(file_pattern)

        results = []
        for file_path in json_files:
            reader = RawDataReader(file_path)
            file_results = {
                'id': reader.id,
                'file_identifier': reader.file_identifier,
                'plan_details': reader.get_plan_details()
            }
            results.append(file_results)
        return results

# Usage
directory_path = '/home/shahpouriz/Data/DBP_newDATA/DBP'
processor = DirectoryProcessor(directory_path)
all_results = processor.process_files()

# for result in all_results:
#     print(result)

------------
# convert_dicom_to_nrrd

In [58]:
import os
import SimpleITK as sitk

class DICOMtoNRRDConverter:
    def __init__(self, directory_path, output_directory):
        self.directory_path = directory_path
        self.output_directory = output_directory
        self.directory_processor = DirectoryProcessor(directory_path)

    def convert_and_save(self):
        patient_data = self.directory_processor.process_files()

        for patient in patient_data:
            patient_id = patient['id']
            output_path = os.path.join(self.output_directory, patient_id)
            if not os.path.exists(output_path):
                os.makedirs(output_path)

            for plan in patient['plan_details']:
                # Processing planningCT file
                if 'planningCT_filename' in plan:
                    nrrd_path = self._process_dose_file(plan['planningCT_filename'], output_path)
                    plan['planningCT_filename'] = nrrd_path or plan['planningCT_filename']
                
                # Processing repeatedCT files
                for exam in plan.get('evaluation_examinations', []):
                    if 'repeatedCT_filename' in exam:
                        nrrd_path = self._process_dose_file(exam['repeatedCT_filename'], output_path)
                        exam['repeatedCT_filename'] = nrrd_path or exam['repeatedCT_filename']
                        
        # Optional: Return the updated patient data for verification
        return patient_data
    

    def _process_dose_file(self, dose_dicom_filename, output_path):
        full_dose_path = os.path.join(self.directory_path, dose_dicom_filename)  # Full path to the DICOM file
        path, name = os.path.split(full_dose_path)
        name, _ = os.path.splitext(name)
        nrrd_filename = name + '.nrrd'
        nrrd_path = os.path.join(output_path, nrrd_filename)

        if not os.path.exists(full_dose_path):
            print(f"File not found: {full_dose_path}")
            return None

        dose_img = sitk.ReadImage(full_dose_path)
        sitk.WriteImage(dose_img, nrrd_path)
        return nrrd_filename


# Usage
directory_path = '/home/shahpouriz/Data/DBP_newDATA/DBP'
output_directory = '/home/shahpouriz/Data/DBP_newDATA/DBP/nrrd'
converter = DICOMtoNRRDConverter(directory_path, output_directory)
updated_patient_data = converter.convert_and_save()

# Print the updated patient data to verify the changes
for patient in updated_patient_data:
    print(patient)

{'id': 'DBP_OP029', 'file_identifier': 'DBP_OP029_Photons_20231013_171408', 'plan_details': [{'plan_name': 'A1FHH', 'planning_exam_name': 'pCTf0', 'planningCT_filename': 'rtdose_pCTf0_1.2.752.243.1.1.20231013171656331.4137.59501.nrrd', 'evaluation_examinations': [{'final_translation_coordinate': {'x': -0.02781, 'y': -0.00155, 'z': -0.0003}, 'repeatedCT_filename': 'rtdose_rCTp1_1.2.752.243.1.1.20231013171733550.7668.47678.nrrd'}, {'final_translation_coordinate': {'x': 0.12961, 'y': -1.08474, 'z': 0.95228}, 'repeatedCT_filename': 'rtdose_rCTp1_1.2.752.243.1.1.20231013172648657.7342.19826.nrrd'}, {'final_translation_coordinate': {'x': -0.41452, 'y': 0.76263, 'z': -1.43247}, 'repeatedCT_filename': 'rtdose_rCTp1_1.2.752.243.1.1.20231013174916210.1228.44072.nrrd'}, {'final_translation_coordinate': {'x': -1.43559, 'y': 0.35916, 'z': -0.14266}, 'repeatedCT_filename': 'rtdose_rCTp1_1.2.752.243.1.1.20231013182427829.9247.24010.nrrd'}, {'final_translation_coordinate': {'x': -0.83831, 'y': -0.8618

In [60]:
import os
import SimpleITK as sitk

class DICOMtoNRRDConverter:
    def __init__(self, directory_path, output_directory_proton, output_directory_foton):
        self.directory_path = directory_path
        self.output_directory_proton = output_directory_proton
        self.output_directory_foton = output_directory_foton
        self.directory_processor = DirectoryProcessor(directory_path)

    def convert_and_save(self):
        patient_data = self.directory_processor.process_files()

        for patient in patient_data:
            patient_id = patient['id']

            for plan in patient['plan_details']:
                # Determine the output path based on whether the plan is for proton or foton
                output_path = self.output_directory_foton if 'F' in plan['plan_name'] else self.output_directory_proton
                full_output_path = os.path.join(output_path, patient_id)
                if not os.path.exists(full_output_path):
                    os.makedirs(full_output_path)

                # Process files for this plan
                nrrd_path = self._process_dose_file(plan['planningCT_filename'], full_output_path)
                plan['planningCT_filename'] = nrrd_path or plan['planningCT_filename']
                
                for exam in plan.get('evaluation_examinations', []):
                    nrrd_path = self._process_dose_file(exam['repeatedCT_filename'], full_output_path)
                    exam['repeatedCT_filename'] = nrrd_path or exam['repeatedCT_filename']

        # Optional: Return the updated patient data for verification
        return patient_data

    def _process_dose_file(self, dose_dicom_filename, output_path):
        full_dose_path = os.path.join(self.directory_path, dose_dicom_filename)  # Full path to the DICOM file
        path, name = os.path.split(full_dose_path)
        name, _ = os.path.splitext(name)
        nrrd_filename = name + '.nrrd'
        nrrd_path = os.path.join(output_path, nrrd_filename)

        if not os.path.exists(full_dose_path):
            print(f"File not found: {full_dose_path}")
            return None

        dose_img = sitk.ReadImage(full_dose_path)
        sitk.WriteImage(dose_img, nrrd_path)
        return nrrd_filename


# Usage
directory_path = '/home/shahpouriz/Data/DBP_newDATA/DBP'
output_directory = '/home/shahpouriz/Data/DBP_newDATA/DBP/nrrd/proton'
converter = DICOMtoNRRDConverter(directory_path, output_directory)
updated_patient_data = converter.convert_and_save()

# Print the updated patient data to verify the changes
for patient in updated_patient_data:
    print(patient)

{'id': 'DBP_OP029', 'file_identifier': 'DBP_OP029_Photons_20231013_171408', 'plan_details': [{'plan_name': 'A1FHH', 'planning_exam_name': 'pCTf0', 'planningCT_filename': 'DBP_OP029/A1FHH/rtdose_pCTf0_1.2.752.243.1.1.20231013171656331.4137.59501.dcm', 'evaluation_examinations': [{'final_translation_coordinate': {'x': -0.02781, 'y': -0.00155, 'z': -0.0003}, 'repeatedCT_filename': 'DBP_OP029/A1FHH/rCTp1/optimization_case_0/rtdose_rCTp1_1.2.752.243.1.1.20231013171733550.7668.47678.dcm'}, {'final_translation_coordinate': {'x': 0.12961, 'y': -1.08474, 'z': 0.95228}, 'repeatedCT_filename': 'DBP_OP029/A1FHH/rCTp1/optimization_case_1/rtdose_rCTp1_1.2.752.243.1.1.20231013172648657.7342.19826.dcm'}, {'final_translation_coordinate': {'x': -0.41452, 'y': 0.76263, 'z': -1.43247}, 'repeatedCT_filename': 'DBP_OP029/A1FHH/rCTp1/optimization_case_2/rtdose_rCTp1_1.2.752.243.1.1.20231013174916210.1228.44072.dcm'}, {'final_translation_coordinate': {'x': -1.43559, 'y': 0.35916, 'z': -0.14266}, 'repeatedCT_f

In [61]:
class DICOMtoNRRDConverter:
    def __init__(self, directory_path, output_directory_proton, output_directory_foton):
        self.directory_path = directory_path
        self.output_directory_proton = output_directory_proton
        self.output_directory_foton = output_directory_foton
        self.directory_processor = DirectoryProcessor(directory_path)

    def convert_and_save(self):
        patient_data = self.directory_processor.process_files()

        for patient in patient_data:
            patient_id = patient['id']

            for plan in patient['plan_details']:
                # Determine the output path based on whether the plan is for proton or foton
                output_path = self.output_directory_foton if 'F' in plan['plan_name'] else self.output_directory_proton
                full_output_path = os.path.join(output_path, patient_id)
                if not os.path.exists(full_output_path):
                    os.makedirs(full_output_path)

                # Process files for this plan
                nrrd_path = self._process_dose_file(plan['planningCT_filename'], full_output_path)
                plan['planningCT_filename'] = nrrd_path or plan['planningCT_filename']
                
                for exam in plan.get('evaluation_examinations', []):
                    nrrd_path = self._process_dose_file(exam['repeatedCT_filename'], full_output_path)
                    exam['repeatedCT_filename'] = nrrd_path or exam['repeatedCT_filename']

        # Optional: Return the updated patient data for verification
        return patient_data

    # ... (rest of the class)


In [62]:
directory_path = '/home/shahpouriz/Data/DBP_newDATA/DBP'
output_directory_proton = '/home/shahpouriz/Data/DBP_newDATA/DBP/nrrd/proton'
output_directory_foton = '/home/shahpouriz/Data/DBP_newDATA/DBP/nrrd/foton'

converter = DICOMtoNRRDConverter(directory_path, output_directory_proton, output_directory_foton)
updated_patient_data = converter.convert_and_save()

# Print the updated patient data
for patient in updated_patient_data:
    print(patient)


AttributeError: 'DICOMtoNRRDConverter' object has no attribute '_process_dose_file'

---------
# Extract information regarding nrrd files

In [57]:

# Save updated_patient_data to a JSON file
output_json_file = output_directory + '/nrrd_file_info.json' 
with open(output_json_file, 'w') as file:
    json.dump(updated_patient_data, file, indent=4)

print(f"Updated patient data saved to {output_json_file}")


Updated patient data saved to /home/shahpouriz/Data/DBP_newDATA/DBP/nrrd/proton/nrrd_file_info.json
